In [32]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from pandas import DataFrame as DF
import time
from pdf2str import extract_text_from_pdf
import os
import random 
from glob import glob

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [33]:
def click_button(xpath='',sleep=0):
    button=browser.find_element_by_xpath(xpath)
    button.click()
    time.sleep(sleep)
    
def sendKeys(keys='',xpath='',sleep=0):
    search_txt=browser.find_element_by_xpath(xpath)
    search_txt.send_keys(keys)
    time.sleep(sleep)

In [332]:
options = Options()
options.headless = False

default_content_setting_values={'notifications' : 2}
prefs = {'profile.default_content_setting_values':default_content_setting_values}
options.add_experimental_option('prefs',prefs)

browser=webdriver.Chrome('/Users/ryh/Desktop/crawler/chromedriver',options=options)
scholar_url='https://scholar.google.com/schhp?hl=zh-TW'
browser.get(scholar_url)


# login
click_button(xpath='//*[@id="gs_hdr_act_s"]',sleep=0)
gmail_user = 'xxx@gmail.com'
sendKeys(keys=gmail_user,xpath='//*[@id="identifierId"]',sleep=1)
click_button(xpath='//*[@id="identifierNext"]/span/span',sleep=2)
gmail_password = 'xxx' 
sendKeys(keys=gmail_password,xpath='//*[@id="password"]/div[1]/div/div[1]/input',sleep=2)
click_button(xpath='//*[@id="passwordNext"]/span/span',sleep=3)


In [35]:
# # search paper
# parent_paper="Inferring Air Quality for Station Location Recommendation Based on Urban Big Data"
# sendKeys(keys=parent_paper,xpath='//*[@id="gs_hdr_tsi"]',sleep=0)
# click_button(xpath='//*[@id="gs_hdr_tsb"]',sleep=0)
# click_button(xpath='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[3]',sleep=0)


In [36]:
# # search keywords

# # keywords=['knn','kriging','interpolation','transfer']
# keywords=['transfer']
# keywords_txt=' OR '.join(keywords)

# click_button(xpath='//*[@id="gs_scipsc"]/span[2]',sleep=0)
# sendKeys(keys=keywords_txt,xpath='//*[@id="gs_hdr_tsi"]',sleep=0)
# click_button(xpath='//*[@id="gs_hdr_tsb"]/span/span[1]',sleep=2)



In [37]:
# soup = BeautifulSoup(browser.page_source)
# papers=soup.find('div',{'id':'gs_res_ccl_mid'}).find_all('div',{'class':'gs_r gs_or gs_scl'})
# len(papers)


In [38]:
# paper=papers[0]

In [39]:
# year=paper.find('div',{'class':'gs_a'}).text.split(',')[-1].split('-')[0]
# year=int(year)
# year

In [40]:
# paper_name=paper.find('h3',{'class':'gs_rt'}).find('a').text
# paper_url=paper.find('h3',{'class':'gs_rt'}).find('a')['href']
# paper_name,paper_url

In [41]:
# authors=paper.find('div',{'class':'gs_a'}).find_all('a')
# authors=['https://scholar.google.com'+x['href'] for x in authors]
# author=authors[0] if len(authors)!=0 else 'x'
# author

In [42]:
# cite_num=paper.find(lambda tag: tag.name=="a" and "被引用" in tag.text)
# cite_num=int(cite_num.text.split(' ')[1]) if cite_num!=None else 0
# cite_num


In [43]:
def get_one_paper(paper):
    if paper.find('h3',{'class':'gs_rt'}).find('a')==None:
        paper_name=paper.find('h3',{'class':'gs_rt'}).find('span',{'id':True}).text
        paper_url='x'
    else:
        paper_name=paper.find('h3',{'class':'gs_rt'}).find('a').text
        paper_url=paper.find('h3',{'class':'gs_rt'}).find('a')['href']
    authors=paper.find('div',{'class':'gs_a'}).find_all('a')
    authors=['https://scholar.google.com'+x['href'] for x in authors]
    author=authors[0] if len(authors)!=0 else 'x'
    cite_num=paper.find(lambda tag: tag.name=="a" and "被引用" in tag.text)
    cite_num=int(cite_num.text.split(' ')[1]) if cite_num!=None else 0
    valid_year = re.compile(r'\b(1[0-9][0-9][0-9]|20[0-9][0-9])\b')
    year=paper.find('div',{'class':'gs_a'}).text.split(',')[-1]
    year=valid_year.search(year)
    year=year.group() if year!=None else 0
    year=int(year)
    return [paper_name,paper_url,author,cite_num,year]

# paper=papers[-1]
# get_one_paper(paper)

In [44]:
# # iterate over pages
# soup = BeautifulSoup(browser.page_source)
# tags=soup.find('div',{'id':'gs_nml'}).find_all('a')
# pages=[0]+[int(x.text)-1 for x in tags]

# result=DF()
# for page in pages[:]:
#     print(page)
    
#     # if not 0, switch to next page
#     if page!=0:
#         xpath='//*[@id="gs_n"]/center/table/tbody/tr/td[%s]/a'%(page+2)
#         click_button(xpath=xpath,sleep=2)
        
#     # current page
#     soup = BeautifulSoup(browser.page_source)
#     papers=soup.find('div',{'id':'gs_res_ccl_mid'}).find_all('div',{'class':'gs_r gs_or gs_scl'})

#     for paper in papers[:2]:
#         row=DF([[parent_paper]+get_one_paper(paper)])
#         row.columns=['parent_paper','paper_name','paper_url','scholar_url','cite_num']
#         result=result.append(row)
    
# result



# paper detail

In [45]:
# i=3
# scholar_url=result.iloc[i,list(result.columns).index('scholar_url')]
# browser.get(scholar_url)

In [46]:
# paper_name=result.iloc[i,list(result.columns).index('paper_name')]
# paper_name

In [47]:
def is_more(browser):
    page_source = browser.page_source
    soup = BeautifulSoup(page_source)
    a=soup.find('button',{'class':'gs_btnPD gs_in_ib gs_btn_flat gs_btn_lrge gs_btn_lsu'})
    is_more='disabled' not in str(a)
    return is_more

In [48]:
# while True:
#     if is_more(browser)==False: break
#     more_button=browser.find_element_by_xpath("//*[contains(text(), '顯示更多結果')]")    
#     time.sleep(3)
#     more_button.click()
#     time.sleep(1)

In [49]:
# click_button(xpath="//*[contains(text(), '%s')]"%(paper_name),sleep=2)

In [50]:
def title2content(soup,title=''):
    paper_conference=soup.find("div",{'class':'gsc_vcd_field'}, string=title)
    paper_conference=paper_conference.find_next('div').text if paper_conference !=None else 'x'
    return paper_conference

In [51]:
# soup = BeautifulSoup(browser.page_source)
# titles=['作者','發布日期','會議','期刊']
# [title2content(soup,title=title) for title in titles]

In [52]:
def get_pdf_url(soup):
    pdfs=soup.find_all('div',{'class':"gsc_vcd_title_ggi"})
    if len(pdfs)==None:
        pdf_url='x'
    else:
        pdfs=[x.find('a')['href'] for x in pdfs if 'pdf' in x.find('a')['href']]
        if len(pdfs)!=0: 
            pdf=pdfs[0]
        else:
            pdf='x'
    return pdf

# get_pdf_url(soup)

In [54]:
# titles=['作者','發布日期','會議','期刊','pdf_url']
# for title in titles:
#     result[title]='x'   
    
# for i in range(3):
#     print(i)
#     scholar_url=result.iloc[i,list(result.columns).index('scholar_url')]
#     paper_name=result.iloc[i,list(result.columns).index('paper_name')]
#     if scholar_url=='x': continue
#     contents=get_paper_detail(browser,scholar_url,paper_name)
#     result.iloc[i,list(result.columns).index('作者'):]=contents
 

In [55]:
def get_cite_url(browser,parent_paper):
    browser.get('https://scholar.google.com/schhp?hl=zh-TW')
    time.sleep(1)

    # search paper
    sendKeys(keys=parent_paper,xpath='//*[@id="gs_hdr_tsi"]',sleep=1)
    click_button(xpath='//*[@id="gs_hdr_tsb"]',sleep=0)
    
    href=None
    soup = BeautifulSoup(browser.page_source)
    tag=soup.find(lambda tag: tag.name=='a' and parent_paper.lower() == tag.text.lower())
    if tag!=None:
        tag=tag.find_parent('div').find(lambda tag: tag.name=='a' and '被引用' in tag.text)
        if tag!=None:
            href='https://scholar.google.com'+tag['href'] if tag!=None else None      
    return href

# parent_paper='Inferring air quality for station location recommendation based on urban big data'
# # parent_paper='Time-sensitive Travel Route Recommendations'
# # parent_paper='A User Friendly Traffic Route Guidance System Employing Enhanced Keyword Tool'
# # parent_paper='urban computing'
# cite_url=get_cite_url(browser,parent_paper)
# print(cite_url)


In [56]:
def get_pages(cite_url,keywords):
    browser.get(cite_url)
    time.sleep(1)
    
    # search keywords 
    keywords_txt=' OR '.join(keywords)
    click_button(xpath='//*[@id="gs_scipsc"]/span[2]',sleep=0)
    sendKeys(keys=keywords_txt,xpath='//*[@id="gs_hdr_tsi"]',sleep=1)
    click_button(xpath='//*[@id="gs_hdr_tsb"]/span/span[1]',sleep=2)
    
    soup = BeautifulSoup(browser.page_source)
    tags=soup.find('div',{'id':'gs_nml'})
    if tags==None: 
        tags=soup.find('div',{'class':"gs_r gs_or gs_scl"})
        if tags==None:
            return []
        else:
            return [0]
        
    tags=tags.find_all('a')
    pages=[0]+[int(x.text)-1 for x in tags]
    return pages


# keywords=['knn','kriging','interpolation','transfer']
# pages=get_pages(cite_url,keywords)
# pages


In [57]:
def get_page_url(browser,pages=[]):
    page=1
    xpath='//*[@id="gs_n"]/center/table/tbody/tr/td[%s]/a'%(page+2)
    click_button(xpath=xpath,sleep=2)
    current_url=browser.current_url
    return current_url

# page_url=get_page_url(browser,pages=pages)
# page_url

In [58]:
def get_page_i_url(page_url,i):
    start=page_url.find('start=')+len('start=')
    end=page_url.find('&q',start)
    url=page_url[:start]+'%s'%(i*10)+page_url[end:]
    return url

# page_i_url=get_page_i_url(page_url,4)
# page_i_url

In [59]:
# browser.get(page_i_url)

In [60]:
def get_one_page(parent_paper,result,browser):
    soup = BeautifulSoup(browser.page_source)
    papers=soup.find('div',{'id':'gs_res_ccl_mid'}).find_all('div',{'class':'gs_r gs_or gs_scl'})

    # current page
    soup = BeautifulSoup(browser.page_source)
    papers=soup.find('div',{'id':'gs_res_ccl_mid'}).find_all('div',{'class':'gs_r gs_or gs_scl'})

    for i,paper in enumerate(papers):
#         print(i)
        row=DF([[parent_paper]+get_one_paper(paper)])
        row.columns=['parent_paper','paper_name','paper_url','scholar_url','cite_num','year']
        result=result.append(row)
    return result


In [61]:
def parent2abstract(parent_paper,keywords,browser):
    result=DF()
    cite_url=get_cite_url(browser,parent_paper)
    if cite_url==None: return result
    
    pages=get_pages(cite_url,keywords)        
    if len(pages)==0: 
        return result
    elif len(pages)==1: 
        result=get_one_page(parent_paper,result,browser)
    else:
        page_url=get_page_url(browser,pages=pages)
        for page in pages:
            print('page',page)
            page_i_url=get_page_i_url(page_url,page)
            browser.get(page_i_url)
            time.sleep(2)
            result=get_one_page(parent_paper,result,browser)
    result=result.reset_index(drop=True)    
    return result

# keywords=['knn','kriging','interpolation','transfer']
# parent_paper='Inferring air quality for station location recommendation based on urban big data'
# parent2abstract(parent_paper,keywords,browser)


# select papers

In [503]:
scholar1=pd.read_pickle('./scholar/Hsun-Ping-Hsieh.pickle')
scholar1=scholar1.fillna('x')
scholar2=pd.read_pickle('./scholar/Yu-Zheng-(郑宇).pickle')
scholar2=scholar2.fillna('x')
scholar=pd.concat([scholar1,scholar2],axis=0)

words=['knn','kriging','interpolation']
c=scholar['keywords'].apply(lambda x: any([word in x for word in words]))
scholar=scholar[c]
scholar.shape

(50, 11)

In [527]:
parents=scholar.paper_name.unique().tolist()
parents.append('x')
len(parents)

51

In [516]:
# parents=[
#     'Mining and planning time-aware routes from check-in data',
#     'Exploiting large-scale check-in data to recommend time-sensitive routes']
# parents.append('x')
# parents


In [528]:
# abstracts=DF()
# i=0
# target_num=5000
# depth=0
# keywords=['knn','kriging','interpolation']
# done=[]

In [63]:
# while True:
#     print(' ------------------------ ')
#     print('len(parents)',len(parents))
#     if len(abstracts)>=target_num: break
#     if len(parents)==0: break
#     parent=parents.pop(0)    
    
#     if parent=='x':
#         depth+=1
#         print('depth',depth)
#         whole=abstracts.paper_name.unique()
#         parents=[x for x in whole if x not in done]
#         if len(parents)==0: break
#         parents.append('x')
#         continue
        
#     try:
#         print('download:',parent)
#         one2many=parent2abstract(parent,keywords,browser)
#         abstracts=abstracts.append(one2many)
#         print('abstracts.shape',abstracts.shape,)
#         done.append(parent)
#         i+=1
#     except:
#         print('error',parent)
#         parents.insert(0,parent)
#         break    

In [65]:
# print(depth)

In [582]:
# c=abstracts.duplicated(subset=['parent_paper','paper_name'], keep=False)
# abstracts[c]

In [575]:
# abstracts.shape

(5000, 6)

In [576]:
# abstracts=abstracts.drop_duplicates(subset=['parent_paper','paper_name'], keep='first')
# abstracts.shape

(4997, 6)

In [577]:
# abstracts.to_pickle('abstracts.pickle')

# page rank

In [583]:
# abstracts['parent']=abstracts['paper_name'].apply(lambda x:x.lower())
# abstracts['child']=abstracts['parent_paper'].apply(lambda x:x.lower())


In [586]:
# c=abstracts.duplicated(subset=['parent','child'], keep=False)
# abstracts[c].shape

(608, 8)

In [587]:
# abstracts=abstracts.drop_duplicates(subset=['parent','child'], keep='first')
# abstracts.shape

(4691, 8)

In [588]:
# abstracts.to_pickle('abstracts2.pickle')

In [74]:
# abstracts=pd.read_pickle('abstracts2.pickle')
# abstracts.shape

(4691, 8)

In [75]:
# parents=abstracts.parent.unique().tolist()
# childs=abstracts.child.unique().tolist()
# papers=list(set(childs+parents))
# len(parents),len(childs),len(papers)

(2810, 648, 2822)

In [76]:
# paper2outs={paper:[] for paper in papers}
# for parent in parents:
#     paper2outs[parent]=abstracts[abstracts.parent==parent].child.tolist()

# paper2ins={paper:[] for paper in papers}
# for child in childs:
#     paper2ins[child]=abstracts[abstracts.child==child].parent.tolist()

In [77]:
# loop_num=20
# paper_num=len(papers)
# d=0.8

# paper2old={}
# for paper in papers:
#     paper2old[paper]=1/paper_num
    
# for i in range(loop_num):
# #     print(i)
#     paper2new={}
#     for paper in papers:
        
#         in_papers=paper2ins[paper]
#         step_by_step=[paper2old[in_paper]/len(paper2outs[in_paper]) for in_paper in in_papers]
#         step_by_step=sum(step_by_step)
        
#         random_open=(1/paper_num)
        
#         paper2new[paper]=d*step_by_step+(1-d)*random_open
    
#     paper2old=paper2new
 

In [93]:
# df_rank=DF({'parent':list(paper2old.keys()),'rank':list(paper2old.values())})
# df_rank=df_rank.sort_values(by='rank',ascending=False).reset_index(drop=True)
# df_rank.head()

,parent,rank
0,map-matching for low-sampling-rate gps traject...,0.033469
1,geolife: a collaborative social networking ser...,0.019284
2,u-air: when urban air quality inference meets ...,0.016788
3,recommending friends and locations based on in...,0.015650
4,searching trajectories by locations: an effici...,0.014820


# detail

In [263]:
def get_paper_detail(browser,scholar_url,paper_name):
    browser.get(scholar_url)
    time.sleep(1)  
    i=0
    while True:
        if is_more(browser)==False: break
        more_button=browser.find_element_by_xpath("//*[contains(text(), '顯示更多結果')]")    
        more_button.click()
        time.sleep(3)  
        i+=1
        if i==4: 
            contents=[paper_name]+['too_long']*5
            row=DF([contents])
            row.columns=['parent','作者','發布日期','會議','期刊','pdf_url']
            return row
            
#     xpath="//*[contains(text(), '%s')]"%(paper_name)
    xpath='//*[contains(translate(text(), "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "abcdefghijklmnopqrstuvwxyz"), "%s")]'%(paper_name)
    click_button(xpath=xpath,sleep=2)
    
    soup = BeautifulSoup(browser.page_source)
    
    titles1=['作者','發布日期','會議','期刊']
    contents1=[title2content(soup,title=title) for title in titles1]
    
    pdf_url=get_pdf_url(soup)
    contents2=[paper_name]+contents1+[pdf_url]
    
    titles2=['parent','作者','發布日期','會議','期刊','pdf_url']
    row=DF([contents2])
    row.columns=titles2
    return row
    

# scholar_url=result.loc[12,'scholar_url']
# paper_name=result.loc[12,'parent']
# get_paper_detail(browser,scholar_url,paper_name)


In [176]:
# dummy=abstracts.drop_duplicates(subset=['parent'], keep='first')
# dummy=dummy.drop(columns=['parent_paper', 'paper_name'])
# result=pd.merge(left=df_rank,right=dummy,left_on='parent',right_on='parent',how='left')
# result=result.fillna('x')
result=pd.read_pickle('result.pickle')
result.head(5)

,parent,rank,paper_url,scholar_url,cite_num,year,child
0,map-matching for low-sampling-rate gps traject...,0.033469,https://dl.acm.org/citation.cfm?id=1653820,https://scholar.google.com/citations?user=h3i6...,654,2009,x
1,geolife: a collaborative social networking ser...,0.019284,http://citeseerx.ist.psu.edu/viewdoc/download?...,https://scholar.google.com/citations?user=5EQf...,795,2010,x
2,u-air: when urban air quality inference meets ...,0.016788,https://dl.acm.org/citation.cfm?id=2488188,https://scholar.google.com/citations?user=DJY8...,536,2013,x
3,recommending friends and locations based on in...,0.015650,https://dl.acm.org/citation.cfm?id=1921596,https://scholar.google.com/citations?user=juUc...,481,2011,geolife: a collaborative social networking ser...
4,searching trajectories by locations: an effici...,0.014820,https://dl.acm.org/citation.cfm?id=1807197,https://scholar.google.com/citations?user=krry...,207,2010,x


In [200]:
# i_list=result.index.tolist()[:]
# details=DF()

In [230]:
# errors=[]

In [281]:
# i_list=errors+i_list
# errors=[]

In [333]:
while True:
    print(' ------------------------ ')
    print('len(i_list)',len(i_list))
    
    if len(i_list)==0: break
    i=i_list.pop(0)
    
    parent=result.loc[i,'parent']
    scholar_url=result.loc[i,'scholar_url']
    if scholar_url=='x': 
        print('no scholar_url:',parent)
        continue
        
    try:
        print('download:',parent)
        detail=get_paper_detail(browser,scholar_url,parent)
        details=details.append(detail)
        print('details.shape',details.shape,)
        
    except Exception as e:
        print(e)
        print('error',parent)
        
        errors.append(i)
#         i_list.insert(0,i)
        break    

 ------------------------ 
len(i_list) 1674
download: a hierarchical approach for identifying user activity patterns from mobile phone call detail records
details.shape (414, 6)
 ------------------------ 
len(i_list) 1673
download: direction matching for sparse movement data sets: determining interaction rules in social groups
details.shape (415, 6)
 ------------------------ 
len(i_list) 1672
download: online moving object visualization with geo-referenced data
details.shape (416, 6)
 ------------------------ 
len(i_list) 1671
download: integrating node embeddings and biological annotations for genes to predict disease-gene associations
details.shape (417, 6)
 ------------------------ 
len(i_list) 1670
download: location-based recommendation systems
details.shape (418, 6)
 ------------------------ 
len(i_list) 1669
download: exploratory visualisation of congestion evolutions on urban transport networks
details.shape (419, 6)
 ------------------------ 
len(i_list) 1668
download: knowled

In [278]:
details.shape

(343, 6)

In [279]:
details.to_pickle('details.pickle')

In [340]:
pd.set_option('max_colwidth', -1)


In [341]:
details2=pd.merge(left=result,right=details,on='parent',how='left')
details2=details2.fillna('x')
cols=['parent','rank','year','cite_num','會議','期刊','paper_url']
details2[cols].head(50)
# details2

,parent,rank,year,cite_num,會議,期刊,paper_url
0,map-matching for low-sampling-rate gps trajectories,0.033469,2009.0,654.0,Proceedings of the 17th ACM SIGSPATIAL international conference on advances in geographic information systems,x,https://dl.acm.org/citation.cfm?id=1653820
1,"geolife: a collaborative social networking service among user, location and trajectory.",0.019284,2010.0,795.0,x,IEEE Data Eng. Bull.,http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.165.4216&rep=rep1&type=pdf
2,u-air: when urban air quality inference meets big data,0.016788,2013.0,536.0,Proceedings of the 19th ACM SIGKDD international conference on Knowledge discovery and data mining,x,https://dl.acm.org/citation.cfm?id=2488188
3,recommending friends and locations based on individual location history,0.015650,2011.0,481.0,x,ACM Transactions on the Web (TWEB),https://dl.acm.org/citation.cfm?id=1921596
4,searching trajectories by locations: an efficiency study,0.014820,2010.0,207.0,Proceedings of the 2010 ACM SIGMOD International Conference on Management of data,x,https://dl.acm.org/citation.cfm?id=1807197
5,"urban computing: concepts, methodologies, and applications",0.014735,2014.0,821.0,x,ACM Transactions on Intelligent Systems and Technology (TIST),https://dl.acm.org/citation.cfm?id=2629592
6,a conceptual framework and taxonomy of techniques for analyzing movement,0.011782,2011.0,145.0,x,Journal of Visual Languages & Computing,https://www.sciencedirect.com/science/article/pii/S1045926X11000139
7,recommender systems survey,0.009771,2013.0,1885.0,x,Knowledge-based systems,https://www.sciencedirect.com/science/article/pii/S0950705113001044
8,retrieving k-nearest neighboring trajectories by a set of point locations,0.008852,2011.0,52.0,International Symposium on Spatial and Temporal Databases,x,https://link.springer.com/chapter/10.1007/978-3-642-22922-0_14
9,destination prediction by sub-trajectory synthesis and privacy protection against such prediction,0.008567,2013.0,225.0,2013 IEEE 29th international conference on data engineering (ICDE),x,https://ieeexplore.ieee.org/abstract/document/6544830/


In [343]:
details2[cols][details2.parent.apply(lambda x: any([key in x for key in ['air','infer']]))]

,parent,rank,year,cite_num,會議,期刊,paper_url
2,u-air: when urban air quality inference meets big data,0.016788,2013.0,536.0,Proceedings of the 19th ACM SIGKDD international conference on Knowledge discovery and data mining,x,https://dl.acm.org/citation.cfm?id=2488188
16,inferring gas consumption and pollution emission of vehicles throughout a city,0.005139,2014.0,176.0,Proceedings of the 20th ACM SIGKDD international conference on Knowledge discovery and data mining,x,https://dl.acm.org/citation.cfm?id=2623653
20,forecasting fine-grained air quality based on big data,0.004107,2015.0,161.0,Proceedings of the 21th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining,x,https://dl.acm.org/citation.cfm?id=2788573
31,deriving high-resolution urban air pollution maps using mobile sensor nodes,0.002485,2015.0,129.0,x,Pervasive and Mobile Computing,https://www.sciencedirect.com/science/article/pii/S1574119214001928
35,a cloud-based knowledge discovery system for monitoring fine-grained air quality,0.002281,2014.0,43.0,too_long,too_long,https://pdfs.semanticscholar.org/54a6/98ad7cf82c98cb3bcb6ce38b88c2e2754974.pdf
39,inferring air quality for station location recommendation based on urban big data,0.002191,2015.0,83.0,Proceedings of the 21th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining,x,https://dl.acm.org/citation.cfm?id=2783344
40,inferring social ties between users with human location history,0.002178,2014.0,122.0,x,Journal of Ambient Intelligence and Humanized Computing,https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s12652-012-0117-z&casa_token=kTvoXUMvgToAAAAA:7mNjZ-lbx46zmsLjDeBBng3Oc5q0nTiDl3eI6D1npXZwfAoC4s4z9YDDzgjfxmBnz8n-luQRhv5XnplB_F8
64,"deep air learning: interpolation, prediction, and feature analysis of fine-grained air quality",0.001010,2018.0,31.0,x,"IEEE Transactions on Knowledge and Data Engineering, doi: 10.1109/TKDE.2018.2823740",https://ieeexplore.ieee.org/abstract/document/8333777/
82,inferring air pollution by sniffing social media,0.000733,2014.0,47.0,Proceedings of the 2014 IEEE/ACM International Conference on Advances in Social Networks Analysis and Mining,x,https://dl.acm.org/citation.cfm?id=3191942
101,"fusing observational, satellite remote sensing and air quality model simulated data to estimate spatiotemporal variations of pm2. 5 exposure in china",0.000561,2017.0,20.0,x,Remote Sensing,https://www.mdpi.com/2072-4292/9/3/221
